In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import geopandas as gpd
import pandas as pd

In [2]:
table_name='providence_test'

### Load the Saved Data

In [3]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

In [4]:
from brix import Indicator, Handler
import OpenCity
import PreCompOsmNet
import statsmodels
from statsmodels.distributions.empirical_distribution import ECDF
import geopandas as gpd
import urllib
import json
import requests
import math
import osmnx
from shapely.geometry import Point, shape
import datetime
from scipy import spatial
import numpy as np
import sys
import pandas as pd

### Create and run the Cityscope model

In [5]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

In [6]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df)
H.add_indicators([d, p, m])

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
walkable_housing
walkable_employment
walkable_healthcare
walkable_hospitality
walkable_shopping
Building Mobility System
	 getting graph
Buffer : 200
Doesnt contain external link types
Buffer : 400
getting external roads
	 Simplified from 2143 to 1891 edges and 929 to 750 nodes
	 Pre-computing paths
Init simulation


/Volumes/GoogleDrive/My Drive/OpenCity/CS_Indicators.py:543: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_zones['x_centroid']=[c.x for c in centroids]


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat
Dens Ind: 0:00:00.004674
Prox Ind: 0:00:00.121711
Prox HM: 0:00:00.005395
Starting MM Update
Schedules and locations
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Finished MM Update


In [7]:
H.listen()

Dens Ind: 0:00:00.006552
Prox Ind: 0:00:00.115102
Prox HM: 0:00:00.005807
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.011831
Prox Ind: 0:00:00.200260
Prox HM: 0:00:00.006333
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.046040
Prox Ind: 0:00:00.630448
Prox HM: 0:00:00.009996
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update


KeyboardInterrupt: 